# Multi-Scale Transformer for GHG Emission Forecasting
## Data Preprocessing and Experiments

This notebook demonstrates:
1. Data download and preprocessing
2. Exploratory data analysis
3. Training baseline models (ARIMA, Prophet, LSTM)
4. Training Multi-Scale Transformer
5. Model comparison and evaluation


In [1]:
# Import libraries
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from pathlib import Path

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

%matplotlib inline
%load_ext autoreload
%autoreload 2


## 1. Data Download and Preprocessing


In [ ]:
# Preprocess the data
from utils.preprocess_data import GHGDataPreprocessor

preprocessor = GHGDataPreprocessor(
    raw_data_dir='../data/raw',
    processed_data_dir='../data/processed'
)

preprocessor.process_all()


## 2. Exploratory Data Analysis


In [ ]:
# Load processed data
with open('../data/processed/facility_series.pkl', 'rb') as f:
    facility_series = pickle.load(f)

with open('../data/processed/sector_series.pkl', 'rb') as f:
    sector_series = pickle.load(f)

with open('../data/processed/national_series.pkl', 'rb') as f:
    national_series = pickle.load(f)

facility_metadata = pd.read_csv('../data/processed/facility_metadata.csv')

print(f"Loaded {len(facility_series)} facility time series")
print(f"Loaded {len(sector_series)} sector time series")
print(f"Loaded national time series with {len(national_series)} years")
print(f"\nSectors: {list(sector_series.keys())}")
print(f"Year range: {national_series.index.min()} - {national_series.index.max()}")


In [ ]:
# Plot national-level emissions over time
fig, ax = plt.subplots(figsize=(12, 6))
national_series.plot(ax=ax, marker='o', linewidth=2, markersize=8, color='darkblue')
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Log(CO2e Emissions)', fontsize=12)
ax.set_title('National GHG Emissions (2010-2023)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../results/figures/national_emissions_trend.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"National emissions range: {np.expm1(national_series.min()):.2e} - {np.expm1(national_series.max()):.2e} metric tons CO2e")


In [ ]:
# Plot sector-level emissions
fig, ax = plt.subplots(figsize=(14, 7))

colors = sns.color_palette("husl", len(sector_series))
for (sector, series), color in zip(sector_series.items(), colors):
    ax.plot(series.index, series.values, marker='o', label=sector, linewidth=2, color=color)

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Log(CO2e Emissions)', fontsize=12)
ax.set_title('Sector-Level GHG Emissions (2010-2023)', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10, framealpha=0.9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../results/figures/sector_emissions_trends.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Distribution of facility emissions (sample)
sample_facilities = list(facility_series.items())[:50]

fig, ax = plt.subplots(figsize=(12, 6))
for fid, series in sample_facilities:
    ax.plot(series.index, series.values, alpha=0.3, linewidth=1)

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Log(CO2e Emissions)', fontsize=12)
ax.set_title('Sample Facility-Level Emissions (50 facilities)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../results/figures/facility_emissions_sample.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Total facilities: {len(facility_series)}")
print(f"Average time series length: {np.mean([len(s) for s in facility_series.values()]):.1f} years")


## 3. Train Baseline Models

### 3.1 ARIMA Baseline


In [ ]:
from baselines.arima_model import evaluate_arima_on_multiple_series

print("Training ARIMA baseline on facility-level data...")
print("(This may take a few minutes)")
print()

arima_results = evaluate_arima_on_multiple_series(
    facility_series,
    train_years=(2010, 2019),
    test_years=(2020, 2023),
    max_series=100,  # Evaluate on subset for speed
    auto_arima_search=True,
    seasonal=False
)

print("\nARIMA Results:")
for metric, value in arima_results['overall_metrics'].items():
    print(f"  {metric}: {value:.4f}")

# Save results
import pickle
with open('../results/metrics/arima_results.pkl', 'wb') as f:
    pickle.dump(arima_results, f)
print("\nSaved ARIMA results to results/metrics/arima_results.pkl")


### 3.2 Prophet Baseline


In [ ]:
from baselines.prophet_model import evaluate_prophet_on_multiple_series

print("Training Prophet baseline on facility-level data...")
print("(This may take a few minutes)")
print()

prophet_results = evaluate_prophet_on_multiple_series(
    facility_series,
    train_years=(2010, 2019),
    test_years=(2020, 2023),
    max_series=100,
    growth='linear',
    yearly_seasonality=False
)

print("\nProphet Results:")
for metric, value in prophet_results['overall_metrics'].items():
    print(f"  {metric}: {value:.4f}")

# Save results
with open('../results/metrics/prophet_results.pkl', 'wb') as f:
    pickle.dump(prophet_results, f)
print("\nSaved Prophet results to results/metrics/prophet_results.pkl")


### 3.3 LSTM Baseline

**Note:** LSTM training requires more computation. Run via command line:

```bash
python src/baselines/train_lstm.py --config configs/lstm_config.yaml
```

This will train the LSTM model and save results to `results/checkpoints/` and metrics to TensorBoard logs.


## 4. Train Multi-Scale Transformer

**Note:** MST training is compute-intensive and best done with GPU. Run via command line:

```bash
python src/models/train_mst.py --config configs/mst_config.yaml
```

**Training features:**
- Multi-scale architecture (fine + coarse encoders)
- Cross-scale fusion with attention
- Hierarchical predictions (facility, sector, national)
- Multi-scale loss function
- Early stopping and checkpointing

**Monitor training:**
```bash
tensorboard --logdir results/logs
```

Then open http://localhost:6006 in your browser.


## 5. Model Comparison and Results


In [ ]:
# Create comparison table
comparison_data = {
    'Model': ['ARIMA', 'Prophet', 'LSTM', 'Multi-Scale Transformer'],
    'MAE': [
        arima_results['overall_metrics'].get('MAE', 0),
        prophet_results['overall_metrics'].get('MAE', 0),
        0,  # To be filled from LSTM results file
        0   # To be filled from MST results file
    ],
    'RMSE': [
        arima_results['overall_metrics'].get('RMSE', 0),
        prophet_results['overall_metrics'].get('RMSE', 0),
        0,
        0
    ],
    'sMAPE': [
        arima_results['overall_metrics'].get('sMAPE', 0),
        prophet_results['overall_metrics'].get('sMAPE', 0),
        0,
        0
    ],
    'MASE': [
        arima_results['overall_metrics'].get('MASE', 0),
        prophet_results['overall_metrics'].get('MASE', 0),
        0,
        0
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("\nModel Comparison (Classical Baselines):")
print("="*60)
print(comparison_df[comparison_df['Model'].isin(['ARIMA', 'Prophet'])].to_string(index=False))
print("="*60)

# Save to CSV
comparison_df.to_csv('../results/metrics/model_comparison.csv', index=False)
print("\nSaved comparison to results/metrics/model_comparison.csv")

# Note about LSTM and MST
print("\nNote: LSTM and MST results will be added after training completes.")
print("Run the training scripts and then use src/utils/evaluate_models.py to generate full comparison.")


In [ ]:
# Visualize model comparison (classical baselines)
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
metrics = ['MAE', 'RMSE', 'sMAPE', 'MASE']

# Only show ARIMA and Prophet for now
baseline_df = comparison_df[comparison_df['Model'].isin(['ARIMA', 'Prophet'])]

for i, (ax, metric) in enumerate(zip(axes.flat, metrics)):
    bars = ax.bar(baseline_df['Model'], baseline_df[metric], alpha=0.7, 
                   color=['#1f77b4', '#ff7f0e'])
    ax.set_title(f'{metric} Comparison', fontsize=12, fontweight='bold')
    ax.set_ylabel(metric, fontsize=10)
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_xticklabels(baseline_df['Model'], rotation=45, ha='right')
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        if height > 0:
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}',
                   ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('../results/figures/baseline_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved comparison plot to results/figures/baseline_comparison.png")


In [ ]:
# Visualize sample predictions
# Get a sample facility's predictions from ARIMA results
if len(arima_results['series_results']) > 0:
    sample_id = list(arima_results['series_results'].keys())[0]
    sample_result = arima_results['series_results'][sample_id]
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    years = np.arange(2020, 2020 + len(sample_result['actuals']))
    ax.plot(years, sample_result['actuals'], marker='o', label='Actual', 
            linewidth=2, markersize=8)
    ax.plot(years, sample_result['predictions'], marker='s', label='ARIMA Prediction', 
            linewidth=2, markersize=8, alpha=0.7)
    
    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Log(CO2e Emissions)', fontsize=12)
    ax.set_title(f'Sample Facility Predictions (Facility {sample_id})', 
                 fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../results/figures/sample_predictions.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Sample facility MAE: {sample_result['metrics']['MAE']:.4f}")
else:
    print("No predictions available to visualize.")


## 6. Summary Statistics


In [ ]:
print("="*70)
print("PROJECT SUMMARY")
print("="*70)
print(f"\n📊 Data Statistics:")
print(f"  - Facilities: {len(facility_series)}")
print(f"  - Sectors: {len(sector_series)}")
print(f"  - Sector names: {', '.join(sector_series.keys())}")
print(f"  - Years: {national_series.index.min()} - {national_series.index.max()}")
print(f"  - Total national emissions range: {np.expm1(national_series.min()):.2e} - {np.expm1(national_series.max()):.2e} metric tons CO2e")

print(f"\n🎯 Models Evaluated:")
print(f"  ✓ ARIMA: Trained on {arima_results['num_series']} facilities")
print(f"  ✓ Prophet: Trained on {prophet_results['num_series']} facilities")
print(f"  ⏳ LSTM: Run via command line (python src/baselines/train_lstm.py)")
print(f"  ⏳ Multi-Scale Transformer: Run via command line (python src/models/train_mst.py)")

print(f"\n🏆 Best Performing Model (Classical Baselines):")
best_model_idx = comparison_df[comparison_df['Model'].isin(['ARIMA', 'Prophet'])]['MAE'].idxmin()
best_model = comparison_df.loc[best_model_idx, 'Model']
best_mae = comparison_df.loc[best_model_idx, 'MAE']
print(f"  - {best_model} (MAE: {best_mae:.4f})")

print(f"\n📁 Results Saved To:")
print(f"  - Metrics: results/metrics/")
print(f"  - Figures: results/figures/")
print(f"  - Comparison table: results/metrics/model_comparison.csv")

print(f"\n📝 Next Steps:")
print(f"  1. Train LSTM: python src/baselines/train_lstm.py --config configs/lstm_config.yaml")
print(f"  2. Train MST: python src/models/train_mst.py --config configs/mst_config.yaml")
print(f"  3. Generate full comparison: python src/utils/evaluate_models.py")
print(f"  4. Fill in interim report: report/interim_report_template.md")
print(f"  5. Create presentation slides using: report/presentation_outline.md")

print("\n" + "="*70)


## 7. Additional Analysis (Optional)

Below are some optional analysis cells you can run to gain more insights:


In [ ]:
# Distribution of emissions by sector
sector_totals = {sector: np.expm1(series.mean()) for sector, series in sector_series.items()}

fig, ax = plt.subplots(figsize=(10, 6))
sectors = list(sector_totals.keys())
totals = list(sector_totals.values())

bars = ax.barh(sectors, totals, alpha=0.7, color=sns.color_palette("husl", len(sectors)))
ax.set_xlabel('Average Annual CO2e Emissions (metric tons)', fontsize=12)
ax.set_title('Average Emissions by Sector (2010-2023)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Add value labels
for i, (bar, value) in enumerate(zip(bars, totals)):
    ax.text(value, i, f'{value:.2e}', va='center', fontsize=9, 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

plt.tight_layout()
plt.savefig('../results/figures/sector_emissions_distribution.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Emission trends (percentage change from 2010 to 2023)
print("Emission Trends (2010 to 2023):\n")
print(f"{'Level':<30} {'2010 Emissions':<20} {'2023 Emissions':<20} {'% Change':<15}")
print("="*85)

# National trend
nat_2010 = np.expm1(national_series[2010])
nat_2023 = np.expm1(national_series[2023])
nat_change = ((nat_2023 - nat_2010) / nat_2010) * 100
print(f"{'National':<30} {nat_2010:<20.2e} {nat_2023:<20.2e} {nat_change:>+14.2f}%")

# Sector trends
for sector, series in sector_series.items():
    if 2010 in series.index and 2023 in series.index:
        sec_2010 = np.expm1(series[2010])
        sec_2023 = np.expm1(series[2023])
        sec_change = ((sec_2023 - sec_2010) / sec_2010) * 100
        print(f"{sector:<30} {sec_2010:<20.2e} {sec_2023:<20.2e} {sec_change:>+14.2f}%")


---

## ✅ Notebook Complete!

You've successfully:
1. ✅ Downloaded/generated EPA GHGRP data
2. ✅ Preprocessed data into hierarchical time series
3. ✅ Performed exploratory data analysis with visualizations
4. ✅ Trained ARIMA and Prophet baseline models
5. ✅ Generated comparison metrics and plots

### 📊 Generated Files:
- **Data:** `data/processed/*.pkl` (facility, sector, national time series)
- **Metrics:** `results/metrics/model_comparison.csv`
- **Figures:** `results/figures/*.png` (6+ publication-quality plots)

### 🚀 Next Steps:

**1. Train Deep Learning Models:**
```bash
# LSTM
python src/baselines/train_lstm.py --config configs/lstm_config.yaml

# Multi-Scale Transformer
python src/models/train_mst.py --config configs/mst_config.yaml
```

**2. Generate Full Model Comparison:**
```bash
python src/utils/evaluate_models.py
```

**3. Complete Interim Report:**
- Open `report/interim_report_template.md`
- Fill in [INSERT] placeholders with your results
- Add figures from `results/figures/`

**4. Create Presentation:**
- Use `report/presentation_outline.md` as guide
- Create 15-16 slides with diagrams and results

**5. Monitor Training (optional):**
```bash
tensorboard --logdir results/logs
```

### 📚 Documentation:
- **Project Summary:** `PROJECT_SUMMARY.md`
- **Full README:** `README.md`
- **Report Template:** `report/interim_report_template.md`

---

**Good luck with your interim report! 🎓**


In [ ]:
# Download EPA GHGRP data
from utils.download_data import EPADataDownloader

downloader = EPADataDownloader(output_dir='../data/raw')

# Try to download real data, or create sample data
print("Attempting to download EPA data...")
# data = downloader.download_all()
data = None

if not data:
    print("Creating sample data for demonstration...")
    data = downloader.create_sample_data()

print(f"\nDownloaded {len(data)} datasets")
